In [5]:
import pandas as pd # algebraic computations , read and write to csv 

import numpy as np # linear algebra and arrays muniplication 

import requests, zipfile, io    # Needed to download and extract the data sets of dev and eval 

# from pandarallel import pandarallel       # Parallel computation on data frames
import time                               # measure excution time of some functions

############  importing the data visualization libraries :##############

import matplotlib.pyplot as plt
import seaborn as sns

############  Import Classifiers #################

from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegressionCV
from sklearn.naive_bayes import BernoulliNB

# Importing a model to split the training set from the evaluation set
from sklearn.model_selection import train_test_split 

############  Evaluation Metric  #################
from sklearn.metrics import f1_score , make_scorer

############  GridSearchCV #################
from sklearn.model_selection import GridSearchCV

################## Tuxtual data prprocessing ##################
from sklearn. feature_extraction.text import TfidfVectorizer 
import warnings
warnings.filterwarnings("ignore")
from os import path
from PIL import Image

import nltk
from wordcloud import WordCloud
from nltk import pos_tag
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer ,word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords as sw
import re
import string
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
#### some libraries must be downloaded
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('pos_tag')
nltk.download('averaged_perceptron_tagger') 

%matplotlib inline
plt.style.use('seaborn-notebook' ) # plotting style

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\musta\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\musta\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Error loading pos_tag: Package 'pos_tag' not found in
[nltk_data]     index
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\musta\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


### download and extract the data sets and make data frames for development and evaluation 

In [ ]:
#############
def download_dataset(zip_file_url):
    r = requests.get(zip_file_url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall()
    
download_dataset('https://dbdmg.polito.it/dbdmg_web/wp-content/uploads/2021/12/DSL2122_january_dataset.zip')



In [ ]:
##### create data frames from the CSV files 

dev_df = pd.read_csv('./DSL2122_january_dataset/development.csv')    
eval_df = pd.read_csv('./DSL2122_january_dataset/evaluation.csv')